# RAG 基礎入門

本範例展示：
1. **第1個儲存格**：建立向量資料庫
2. **第2個儲存格**：查詢向量資料庫

學習目標：理解如何將文本轉換為向量並進行相似度檢索

In [1]:
# 第1個儲存格：建立向量資料庫

import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb

# 定義包含文字檔案的目錄和持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
file_path = os.path.join(current_dir, "books", "智慧型手機使用手冊.txt")
persistent_directory = os.path.join(current_dir, "db", "chroma_db_chinese_nb")
print(persistent_directory)

# 檢查 Chroma 向量存儲是否已存在
if not os.path.exists(persistent_directory):
    print("持久化目錄不存在。正在初始化向量存儲...")

    # 確保文字檔案存在
    if not os.path.exists(file_path):
        raise FileNotFoundError(
            f"檔案 {file_path} 不存在。請檢查路徑。"
        )

    # 從檔案讀取文字內容
    loader = TextLoader(file_path)
    documents = loader.load()

    # 將文件分割成塊
    # chunk_size=1000: 每個文本區塊最多 1000 個字元
    # chunk_overlap=0: 區塊之間不重疊
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    # 顯示分割文件的資訊
    print("\n--- 文件塊資訊 ---")
    print(f"文件塊數量: {len(docs)}")
    print(f"範例塊:\n{docs[0].page_content}\n")

    # 建立嵌入模型
    print("\n--- 正在建立嵌入 ---")
    print("使用 Jina Embeddings v2（繁體中文開源模型）")
    embeddings = HuggingFaceEmbeddings(
        model_name="jinaai/jina-embeddings-v2-base-zh"
    )
    print("\n--- 完成建立嵌入 ---")

    # 建立向量存儲並自動持久化
    print("\n--- 正在建立向量存儲 ---")
    
    # 使用 PersistentClient 以避免權限問題
    client = chromadb.PersistentClient(path=persistent_directory)
    
    db = Chroma.from_documents(
        docs, 
        embeddings, 
        client=client,
        collection_name="smartphone_manual"
    )
    print("\n--- 完成建立向量存儲 ---")

else:
    print("向量存儲已存在。無需初始化。")

/Users/roberthsu2003/Documents/GitHub/langchain_project/4_rag/db/chroma_db_chinese_nb
向量存儲已存在。無需初始化。


In [2]:
# 第2個儲存格：查詢向量資料庫

import os
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
import chromadb

# 定義持久化目錄
current_dir = os.path.dirname(os.path.abspath("__file__"))
persistent_directory = os.path.join(current_dir, "db", "chroma_db_chinese_nb")

# 定義嵌入模型（使用開源繁體中文模型）
embeddings = HuggingFaceEmbeddings(model_name="jinaai/jina-embeddings-v2-base-zh")

# 使用 PersistentClient 載入現有的向量存儲
client = chromadb.PersistentClient(path=persistent_directory)

db = Chroma(
    client=client,
    collection_name="smartphone_manual",
    embedding_function=embeddings
)

# 定義使用者的問題
query = "如何設定指紋辨識？"

# 根據查詢檢索相關文件
# search_type="similarity": 使用相似度搜尋（預設方法）
# k=3: 返回最相關的 3 個文件
# 注意：由於某些 embedding 模型的相似度分數可能不在 0-1 範圍內，
#      因此使用簡單的 similarity 搜尋而非 similarity_score_threshold
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3},
)
relevant_docs = retriever.invoke(query)

# 顯示相關結果及元數據
print("\n--- 相關文件 ---")
for i, doc in enumerate(relevant_docs, 1):
    print(f"文件 {i}:\n{doc.page_content}\n")
    if doc.metadata:
        print(f"來源: {doc.metadata.get('source', 'Unknown')}\n")

Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-zh and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.dense.weight', 'encoder.layer.1.output.LayerNorm.bias', 'encoder.layer.1.output.LayerNorm.weight', 'encoder.layer.1.output.dense.bias', 'encoder.layer.1.output.dense.weight', 'encoder.layer.10.intermediate.dense.bias', 'encoder.layer.10.intermediate.dense.weight', 'encoder.layer.10.output.LayerNorm.bias', 'encoder.layer.10.output.LayerNorm.weight', 'encoder.layer.10.output.dense.bias', 'encoder.layer.10.output.dense.weight', 'encoder.layer.11.intermediate.dense.bias', 'encoder.layer.11.intermedi


--- 相關文件 ---
文件 1:
前置鏡頭:
- 3200 萬畫素(f/2.0)
- 支援人像模式與美顏功能

拍照模式:
- 自動模式:智慧場景辨識
- 人像模式:背景虛化效果
- 夜景模式:低光源拍攝
- 專業模式:手動調整 ISO、快門、白平衡
- 全景模式:180° 廣角拍攝
- 慢動作:120fps/240fps 錄影

3.2 相機進階功能
AI 場景辨識:
自動識別 20+ 種場景(美食、風景、人物、寵物等)並優化參數

HDR(高動態範圍):
同時拍攝多張不同曝光的照片並合成,保留更多亮部與暗部細節

夜景模式使用技巧:
1. 在光線不足環境下,相機會自動建議使用夜景模式
2. 保持手機穩定 3-5 秒
3. 手機會自動拍攝多張照片並合成
4. 避免手震可使用腳架或穩定支撐

3.3 錄影功能
支援解析度:
- 4K @ 60fps
- 1080p @ 30/60/120fps
- 720p @ 30fps

錄影特色功能:
- 即時美顏錄影
- 電影模式(淺景深效果)
- 縮時攝影
- 慢動作錄影
- 超穩定防震技術

第四章:電池與充電

4.1 電池規格
電池容量:5000mAh
正常使用續航力:
- 通話時間:最長 28 小時
- 影片播放:最長 18 小時
- 網頁瀏覽:最長 16 小時
- 待機時間:最長 450 小時

4.2 充電方式
有線充電:
- 支援 65W 快速充電
- 30 分鐘可充至 70%
- 完整充電約 45 分鐘
- 建議使用原廠充電器

無線充電:
- 支援 15W Qi 無線充電
- 相容市面上大多數無線充電板
- 完整充電約 2.5 小時

反向無線充電:
可將手機作為無線充電板,為其他支援無線充電的裝置充電(如無線耳機)

4.3 省電建議
延長電池續航力的方法:
1. 降低螢幕亮度或使用自動亮度
2. 縮短螢幕自動關閉時間(設定於 30 秒-1 分鐘)
3. 關閉不使用的藍牙、Wi-Fi、GPS
4. 啟用省電模式或超級省電模式
5. 限制背景應用程式活動
6. 關閉應用程式通知推送
7. 使用深色主題(OLED 螢幕特別有效)

電池保養建議:
- 避免在極端溫度環境下使用(建議 0-35°C)
- 不要等到 0% 才充電,建議保持 20-80% 之間
- 避免整夜充電或長時間維持 100%
- 每

In [4]:
# 第3個儲存格：整合 Chain - 建立簡單的 RAG 問答鏈

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import ChatOllama

# 定義使用者的問題
question = "如何設定指紋辨識？"

print(f"問題: {question}\n")

# 步驟 1: 從向量資料庫檢索相關文件
print("=" * 60)
print("步驟 1: 從向量資料庫檢索相關文件")
print("=" * 60)

retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}  # 只取最相關的 2 個文件
)
retrieved_docs = retriever.invoke(question)

print(f"找到 {len(retrieved_docs)} 個相關文件\n")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"文件 {i} (前100字):")
    print(doc.page_content[:100] + "...\n")

# 步驟 2: 建立 RAG Chain
print("=" * 60)
print("步驟 2: 建立 RAG Chain")
print("=" * 60)

# 定義提示模板
template = """你是一個智慧型手機的客服助手。請根據以下參考資料回答使用者的問題。

參考資料：
{context}

使用者問題：{question}

請用繁體中文回答，並且：
1. 只根據參考資料回答，不要編造內容
2. 如果參考資料中沒有答案，請誠實說「我在資料中找不到相關資訊」
3. 回答要清楚、具體、有條理

回答："""

prompt = ChatPromptTemplate.from_template(template)

# 建立 LLM（使用本地 Ollama）
llm = ChatOllama(model="llama3.2", temperature=0)

# 定義文件格式化函數
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 建立完整的 RAG Chain
# RunnablePassthrough() 讓 question 直接傳遞下去
# retriever | format_docs 將檢索結果格式化為 context
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG Chain 建立完成\n")

# 步驟 3: 執行 RAG Chain 並取得答案
print("=" * 60)
print("步驟 3: 執行 RAG Chain")
print("=" * 60)

answer = rag_chain.invoke(question)

print(f"\n【AI 回答】\n{answer}\n")

print("=" * 60)
print("RAG Chain 流程總結")
print("=" * 60)
print("1. 使用者提問 → 向量檢索找相關文件")
print("2. 將文件和問題組合成提示詞")
print("3. 送給 LLM 生成答案")
print("4. 解析並返回答案")
print("\n💡 這就是從 Chain 3 學到的技巧應用在 RAG 上！")

問題: 如何設定指紋辨識？

步驟 1: 從向量資料庫檢索相關文件
找到 2 個相關文件

文件 1 (前100字):
前置鏡頭:
- 3200 萬畫素(f/2.0)
- 支援人像模式與美顏功能

拍照模式:
- 自動模式:智慧場景辨識
- 人像模式:背景虛化效果
- 夜景模式:低光源拍攝
- 專業模式:手動調整 IS...

文件 2 (前100字):
硬體規格:
- 處理器:Snapdragon 8 Gen 2 / Apple A17 Pro
- 記憶體:8GB/12GB RAM
- 儲存空間:128GB/256GB/512GB
- 螢幕:6.7"...

步驟 2: 建立 RAG Chain
✅ RAG Chain 建立完成

步驟 3: 執行 RAG Chain

【AI 回答】
根據參考資料，我們可以知道如何設定指紋辨識：

1. 首先，開啟手機的設定菜單。
2. 找到「生物認證」或「指紋辨識」選項，通常位於「安全性」或「保護模式»下的選項中。
3. 選擇「指紋辨識」並確定您想啟用指紋辨識功能。
4. 在指紋辨識設定頁面上，您需要將您的指紋添加到手機中。請將左手或右手的指紋添加到手機中，依序排列指尖以確定您的指紋。
5. 一旦您完成指紋辨識設定，手機就會記住您的指紋，並允許您使用指紋辨識來開啟手機、鎖屏或進行其他安全功能。

請注意，如果您在資料中找不到相關資訊，我們可以誠實地說「我在資料中找不到相關資訊」。

RAG Chain 流程總結
1. 使用者提問 → 向量檢索找相關文件
2. 將文件和問題組合成提示詞
3. 送給 LLM 生成答案
4. 解析並返回答案

💡 這就是從 Chain 3 學到的技巧應用在 RAG 上！
